# Image Processing Final Project
This notebook includes solutions to all 6 problems using Python and OpenCV.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

def show(image, title='', cmap='gray'):
    plt.imshow(image, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

## Problem 1

In [ ]:
img = cv2.imread('tf2_engineer.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
show(img_rgb, "Original Image", cmap=None)

h, w, _ = img.shape
yc, xc = h // 2, w // 2
print(f"Center coordinates: ({yc}, {xc}) - Pixel: {img[yc, xc]}")

color_patch = (168, 158, 50)  # #329ea8 in BGR
cv2.rectangle(img, (xc-20, yc-15), (xc+20, yc+15), color_patch, -1)

print(f"Pixel at center of patch (BGR): {img[yc, xc]}")
show(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), "Image with Color Patch", cmap=None)

## Problem 2

In [ ]:
img = cv2.imread('einstein.tiff', 0)
show(img, "Original Image")

negative = 255 - img
show(negative, "Negative Image")

print("Random 5 pixel values:")
for _ in range(5):
    y = random.randint(0, img.shape[0]-1)
    x = random.randint(0, img.shape[1]-1)
    print(f"({y}, {x}) -> Original: {img[y, x]}, Negative: {negative[y, x]}")

## Problem 3

In [ ]:
img = cv2.imread('pout.tiff', 0)
show(img, "Original Image")

log_trans = (np.log1p(img)).astype(np.float32)
log_trans = cv2.normalize(log_trans, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
show(log_trans, "Log Transform")

inv_log_trans = (np.expm1(img / 255 * np.log(256))).astype(np.uint8)
show(inv_log_trans, "Inverse Log Transform (Original Image)")

inv_back = (np.expm1(log_trans / 255 * np.log(256))).astype(np.uint8)
show(inv_back, "Inverse of Log-Transformed Image")

print("Log transform enhances darker regions. Inverse of log-transformed image does not perfectly match original.")

## Problem 4

In [ ]:
img = cv2.imread('moon.tiff', 0)
show(img, "Original Image")

blurred = cv2.GaussianBlur(img, (9, 9), 10)
mask = cv2.subtract(img, blurred)

for k in [0.2, 0.5, 1.0]:
    sharpened = cv2.addWeighted(img, 1.0, mask, k, 0)
    show(sharpened, f"Spatial Unsharp Masking (k={k})")

In [ ]:
def high_pass_filter(shape, D0):
    u = np.arange(-shape[0]//2, shape[0]//2)
    v = np.arange(-shape[1]//2, shape[1]//2)
    U, V = np.meshgrid(u, v, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    H = 1 - np.exp(-(D**2) / (2*(D0**2)))
    return H

img = cv2.imread('moon.tiff', 0)
f = np.fft.fftshift(np.fft.fft2(img))

H = high_pass_filter(img.shape, 30)
for k in [0.2, 0.5, 1.0]:
    G = (1 + k * H) * f
    g = np.fft.ifft2(np.fft.ifftshift(G))
    g = np.abs(g)
    g = cv2.normalize(g, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    show(g, f"Frequency Domain Unsharp Masking (k={k})")

## Problem 5

In [ ]:
img = cv2.imread('pcb.tiff', 0)
show(img, "Original Image")

hist = cv2.calcHist([img], [0], None, [256], [0,256])
plt.plot(hist), plt.title("Histogram"), plt.show()

median = cv2.medianBlur(img, 3)
show(median, "Median Filtered")

gaussian = cv2.GaussianBlur(median, (3,3), 1)
show(gaussian, "Median + Gaussian Filtered")

## Problem 6

In [ ]:
img = cv2.imread('pollen.tiff', 0)
show(img, "Original Image")

hist = cv2.calcHist([img], [0], None, [256], [0,256])
plt.plot(hist), plt.title("Histogram"), plt.show()

# Histogram Equalization
equalized = cv2.equalizeHist(img)
show(equalized, "Histogram Equalized")

# CLAHE (Adaptive)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
adaptive_eq = clahe.apply(img)
show(adaptive_eq, "CLAHE Result")